This code was borrowed from AAI 540 labs

https://github.com/mechristenson/aai-540-labs.git

## Create S3 Bucket

In [1]:
pip install --upgrade boto3 botocore awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 191.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 160.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 52.8 MB/s  0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9.1
    Uninstalling rsa-4.9.1:
      Successfully uninstalled rsa-4.9.1
  Attempting uninstall: botocorem━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [docutils]
    Found existing installation: botocore 1.37.1━━━━━━━━━━━━━━ 1/6 [docutils]
    Uninstalling botocore-1.37.1:0m━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [botocore]
      Successfully uninstalled botocore-1.37.1━━━━━━━━━━━━━━━━ 2/6 [botocore]
  Attempting uninstall: s3transferm━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [botocore]
    Found existing installation: s3transfer 0.11.3━━━━━━━━━━━━━━━━ 3/6 [s3transfer]
    Uninstalling s3transfer-0.11.3:━━━━━━━━━━━━━━━━━━━ 3/6 [s3transfer]
      Successfully uninstalled s3transfer-0.11.3━━━━━━━━━━━━━━ 3/6 [s3transfer]
  

In [2]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
setup_s3_bucket_passed = False

In [4]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-590183687297


Verify S3_BUCKET Bucket Creation

In [5]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'ENWCNP07276K5ZSB', 'HostId': 'Tg05tJqfeO8P33BBFygDwHi+FlrvGRUDFvM89uM6DVBFbtCt8GexN0hCRra59gw62Ul3uMECRoM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Tg05tJqfeO8P33BBFygDwHi+FlrvGRUDFvM89uM6DVBFbtCt8GexN0hCRra59gw62Ul3uMECRoM=', 'x-amz-request-id': 'ENWCNP07276K5ZSB', 'date': 'Fri, 12 Sep 2025 01:53:31 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [6]:
%store setup_s3_bucket_passed


Stored 'setup_s3_bucket_passed' (bool)


In [7]:
%store


Stored variables and their in-db values:
setup_s3_bucket_passed             -> True


### Release Resources

In [8]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [1]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

## Set up Data lake

In [7]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


#### Set S3 Destination Folder

In [8]:
s3_private_path_parquet = "s3://{}/amazon-reviews-pds/parquet".format(bucket)
print(s3_private_path_parquet)

s3://sagemaker-us-east-1-590183687297/amazon-reviews-pds/parquet


In [2]:
%store -r setup_s3_bucket_passed

In [4]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")


In [5]:
if not setup_s3_bucket_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

### Copy data from github to S3 bucket

In [9]:
!aws s3 cp --recursive /home/sagemaker-user/aai540_toxicity_classification/civil $s3_private_path_parquet/ --include "*"


upload: civil/validation-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/amazon-reviews-pds/parquet/validation-00000-of-00001.parquet
upload: civil/train-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/amazon-reviews-pds/parquet/train-00000-of-00001.parquet
upload: civil/test-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/amazon-reviews-pds/parquet/test-00000-of-00001.parquet


### List the files

In [10]:
print(s3_private_path_parquet)

s3://sagemaker-us-east-1-590183687297/amazon-reviews-pds/parquet


In [12]:
!aws ls s3 s3_private_path_parquet

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument command: Invalid choice, valid choices are:

accessanalyzer                           | account                                 
acm                                      | acm-pca                                 
aiops                                    | amp                                     
amplify                                  | amplifybackend                          
amplifyuibuilder                         | apigateway                              
apigatewaymanagementapi                  | apigatewayv2                         